<a href="https://colab.research.google.com/github/oopsno/nuse/blob/master/docs/nuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 获取数据

要在 Colab 中尝试运行 oopsno/nuse，首先需要获取 MoNuSeg 数据集。


将以下三个文件转存到 `[你的Google Drive]/nuse/` 目录下以完成数据获取工作。


- https://drive.google.com/open?id=1gER1yzLZu-J1ehLHExfZ5sepFWvl4N3E
- https://drive.google.com/open?id=1-7kj7cYStrlORMjT1DXGX7x0TtVSxQyG
- https://drive.google.com/open?id=1xT1yVbScnEAhSlQunTv2E_LraoS3aqwC

# Colab 环境预备

Colab 环境预备共分三步

- 挂载 Google Drive
- 克隆 oopsno/nuse 并安装依赖 Python
- 配置 Visdom

## 挂载 Google Drive 到 `/content/gdrive`


运行以下代码并按照说明操作

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

##  克隆 oopsno/nuse 并安装依赖

说明：移除 albumentat 是为了解决 Colab 运行时的下预装的 Python 包和 oopsno/nuse 的依赖项之间的冲突。

In [0]:
%%sh
NUSE_ROOT="/content/nuse-root"
git clone https://github.com/oopsno/nuse.git $NUSE_ROOT
ln -s $NUSE_ROOT/nuse /content
yes | pip uninstall albumentations
pip install -r $NUSE_ROOT/requirements.txt

## 配置 Visdom

### 安装 ngrok 

安装 ngrok 以在任何地方访问 visdom。详情参见：https://ngrok.com/

In [0]:
%%sh
if [ ! -f ngrok ] ; then
    wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    unzip ngrok-stable-linux-amd64.zip
fi

### 启动 Visdom 和 Ngrok 并设置隧道

以下代码所打印的结果即可用于直接访问 visdom

In [0]:
import subprocess
import json
import time

visdom_process = subprocess.Popen(['visdom'], stdout=subprocess.DEVNULL)
ngrok_process = subprocess.Popen(['./ngrok', 'http', '8097'], stdout=subprocess.PIPE)

while True:
    try:
        curl_process = subprocess.Popen(['curl', '-s', 'http://localhost:4040/api/tunnels'], stdout=subprocess.PIPE)
        tunnel_json, _ = curl_process.communicate()
        print('Visdom 已启动，请访问:', json.loads(tunnel_json)['tunnels'][0]['public_url'])
    except:
        time.sleep(1) # 等待上述进程启动
    else:
        break

### 终止 Visdom 和 Ngrok

终止 visdom 和 ngrok 有三种方法

- 等待当前代码执行程序因超时而终止
- 手工重启代码执行环境
- 使用以下代码

In [0]:
def terminate_visdom_ngrok():
    global visdom_process, ngrok_process
    try:
        visdom_process.kill()
        ngrok_process.kill()
    except:
        print('未找到进程')

#  准备数据

运行以下代码以从原始数据中准备训练所需的数据并将之缓存于 Google Drice。
这一过程需要长达数分钟的运行时间。
为了缓解等待，以下代码会优先检查 Google Drive 中的缓存。

**注意：修改与数据相关的代码后，请手动删除缓存并重新打包数据，以免产生难以排除的错误。**

In [0]:
%%sh

NUSE_ROOT="/content/nuse-root"
GDRIVE="/content/gdrive/My Drive"
GDRIVE_MONUSEG_PTH="$GDRIVE/nuse/monuseg.pth"
GDRIVE_MONUSEG_ZIP="$GDRIVE/nuse/MoNuSegTrainingData.zip"
LOCAL_PTH="$NUSE_ROOT/monuseg.pth"

if [ -f "$GDRIVE_MONUSEG_PTH" ] ; then
  echo "Using cached '$GDRIVE_MONUSEG_PTH' -> '$LOCAL_PTH'"
  cp "$GDRIVE_MONUSEG_PTH" "$LOCAL_PTH"
else
  echo "Building '$LOCAL_PTH'"
  pip install spams
  cd /content/nuse/MoNuSeg
  python prepare_data.py "$GDRIVE_MONUSEG_ZIP" "$LOCAL_PTH"
  echo "Caching '$LOCAL_PTH' -> '$GDRIVE_MONUSEG_PTH'"
  cp "$LOCAL_PTH" "$GDRIVE_MONUSEG_PTH"
fi

# 启动训练


## 检查运行时

如果当前 Colab 代码执行程序配置正确，运行以下代码将打印可用的 GPU 信息

In [0]:
import torch.cuda

p = None

if torch.cuda.device_count() == 0:
    print('未找到 GPU, 请重新配置 Colab 的代码执行程序')
else:
    for device_id in range(torch.cuda.device_count()):
        p = torch.cuda.get_device_properties(device_id)
        print(f'GPU {device_id}')
        print(f'  NAME = {p.name}')
        print(f'  ARCH = {p.major}.{p.minor}')
        print(f'  GRAM = {p.total_memory}')


## 短期训练

在 Google Colab 提供的免费的 Tesls T4 上实验性的训练模型。


在训练过程中，可以通过之前步骤创建的 visdom 实时地观察训练过程中的各项指标和中间结果的变化。

注意事项：

- 考虑到 Google Colab 对免费 GPU 的限制使用，这里仅仅训练 16 代，大约需要 10 分钟
- 请确保当前代码执行程序的硬件加速器已设置为 **GPU**
- 训练过程默认记录在 visdom 中的 `resunet_dice` 环境下，需要在 visdom 页面顶部手切换
- 文本形式的日志记录在文件 `/content/nuse/nuse.log` 中
- 训练产生的权重文件存储于 Google Drive 的  `/nuse/snapshot` 目录下

In [0]:
%%sh
NUSE_ROOT="/content/nuse-root"
mkdir -p "/content/gdrive/My Drive/nuse/snapshor"
cd $NUSE_ROOT
python train.py --max_epochs 16 --snapshot_dir "/content/gdrive/My Drive/nuse/snapshor" --batch_size 8

## 检查训练结果

运行以下代码以检查经过 16 代训练之后的模型在 MoNuSeg 测试集中的各个样本上取得的 AJI 以及整体均值。

In [0]:
%%sh
grep AJI $ROOT/nuse.log

如果配置正确，即使仅仅经过了十分钟的训练，整体 AJI 也应该在 0.5 以上。

# 可视化

注意：为了获得观感更好的结果，这一部分加载的是经过充分训练的模型。

In [0]:
import torch
import nuse.nn.resunet
import nuse.nn.dpunet
import nuse.monuseg
import nuse.metrics
import nuse.utils.cnn3_decoder
from torchvision.transforms.functional import to_pil_image
from torch.nn.functional import interpolate
import PIL.Image
import numpy as np

WEIGHTS = '/content/gdrive/My Drive/nuse/resunet_dice_pretrained.pth'

def initialize_model():
    # 创建模型并加载权重
    m = nuse.nn.resunet.ResUNet(False).eval()
    m.load_state_dict(torch.load(WEIGHTS, 'cpu'))
    return m

def load_test_data():
    # 加载第一张测试数据
    ds = nuse.monuseg.MoNuSeg('/content/nuse-root/monuseg.pth', testing=True)
    x = ds[0]
    x = x.unsqueeze(0)
    return x

@torch.no_grad()
def inference(m, x):
    h = torch.sigmoid(m(x))
    return h[0, 1:, 12:1012, 12:1012]  # 切除用于对其图片大小的零填充


def decode_inference(h, threshold=0.5):
    boundary, inside = h
    boundary = (boundary > threshold).int()
    inside = (((inside > threshold).int() - boundary) > threshold).byte().numpy()
    boundary = boundary.byte().cpu().numpy()
    inside = nuse.metrics.area_filter(inside, threshold=32)
    inside = cv2.erode(inside, np.array([[0, 1, 1], [1, 1, 1], [0, 1, 1]], dtype=np.uint8))
    decoded = nuse.utils.cnn3_decoder.decode(boundary, inside).astype(np.uint8)
    result = nuse.metrics.area_filter(decoded, threshold=32)
    return result


def visualize(h):
    _, height, width = h.size()
    decoded = decode_inference(h)
    decoded = to_pil_image(torch.from_numpy(decoded).float())
    boundary = to_pil_image((h[0] > 0.5).byte() * 255)
    inside = to_pil_image((h[1] > 0.5).byte() * 255)
    canvas = PIL.Image.new('RGB', (3 * width + 2, height), color=(255, 0, 0))
    canvas.paste(boundary, (0, 0))
    canvas.paste(inside,   (1 * width + 1, 0))
    canvas.paste(decoded,  (2 * width + 2, 0))
    return canvas

可视化运行时间可能较长（以分钟计）

In [0]:
visualize(inference(initialize_model(), load_test_data()))